In [2]:
import os
import numpy as np
import glob
# import torch
# import torchio as tio
from sklearn.model_selection import KFold
from tqdm import tqdm

In [3]:

def get_split(data, idx):
    return list(np.array(data)[idx])


def load_data(path, files_pattern):
    return sorted(glob.glob(os.path.join(path, files_pattern)))


def get_kfold_splitter(nfolds):
    return KFold(n_splits=nfolds, shuffle=True, random_state=12345)


def get_test_fnames(args, data_path, meta=None):
    kfold = get_kfold_splitter(args.nfolds)
    test_imgs = load_data(data_path, "*_x.npy")
    if args.exec_mode == "predict" and "val" in data_path:
        _, val_idx = list(kfold.split(test_imgs))[args.fold]
        test_imgs = sorted(get_split(test_imgs, val_idx))
        if meta is not None:
            meta = sorted(get_split(meta, val_idx))
    return test_imgs, meta

In [33]:
!cd /data/private_data/schw && ls

33_3d  39_3d  schw_1_reg  schw_4a_resamp


In [24]:
nfolds=3
data_path = '/data/newdata/schw/37_3d'
kfold = KFold(n_splits=nfolds, shuffle=True, random_state=12345)
fold=2

In [25]:
meta = load_data(data_path, "*_meta.npy")
orig_lbl = load_data(data_path, "*_orig_lbl.npy")
        
imgs, lbls = load_data(data_path, "*_x.npy"), load_data(data_path, "*_y.npy")
train_idx, val_idx = list(kfold.split(imgs))[fold]
print(len(train_idx)), print(len(val_idx))
orig_lbl, meta = get_split(orig_lbl, val_idx), get_split(meta, val_idx)
train_imgs, train_lbls = get_split(imgs, train_idx), get_split(lbls, train_idx)
val_imgs_4, val_lbls_4 = get_split(imgs, val_idx), get_split(lbls, val_idx)

162
80


In [26]:
lbls_2 = [each.split('/')[-1][:-6] for each in val_lbls_4]

In [27]:
import json
import mpu.io

data = {'fold_0': lbls_0,
        'fold_1': lbls_1,
        'fold_2': lbls_2,
#         'fold_3': val_lbls_3,
#         'fold_4': val_lbls_4,
       }


mpu.io.write('/home/polina/DeepLearningExamples/PyTorch/Segmentation/nnUNet/metrics/schw_seed.json', data)

{'fold_0': ['VS-SEG-010',
  'VS-SEG-011',
  'VS-SEG-015',
  'VS-SEG-017',
  'VS-SEG-018',
  'VS-SEG-019',
  'VS-SEG-020',
  'VS-SEG-021',
  'VS-SEG-027',
  'VS-SEG-028',
  'VS-SEG-031',
  'VS-SEG-034',
  'VS-SEG-042',
  'VS-SEG-049',
  'VS-SEG-051',
  'VS-SEG-052',
  'VS-SEG-054',
  'VS-SEG-056',
  'VS-SEG-058',
  'VS-SEG-059',
  'VS-SEG-068',
  'VS-SEG-069',
  'VS-SEG-070',
  'VS-SEG-073',
  'VS-SEG-075',
  'VS-SEG-087',
  'VS-SEG-091',
  'VS-SEG-094',
  'VS-SEG-095',
  'VS-SEG-096',
  'VS-SEG-102',
  'VS-SEG-103',
  'VS-SEG-106',
  'VS-SEG-112',
  'VS-SEG-113',
  'VS-SEG-115',
  'VS-SEG-120',
  'VS-SEG-123',
  'VS-SEG-125',
  'VS-SEG-131',
  'VS-SEG-132',
  'VS-SEG-141',
  'VS-SEG-144',
  'VS-SEG-145',
  'VS-SEG-152',
  'VS-SEG-154',
  'VS-SEG-156',
  'VS-SEG-157',
  'VS-SEG-163',
  'VS-SEG-164',
  'VS-SEG-165',
  'VS-SEG-166',
  'VS-SEG-171',
  'VS-SEG-173',
  'VS-SEG-178',
  'VS-SEG-179',
  'VS-SEG-180',
  'VS-SEG-183',
  'VS-SEG-185',
  'VS-SEG-193',
  'VS-SEG-194',
  'VS-SEG-196'

In [14]:
# sub_ids
import pandas as pd
data = {'0': sub_ids,
        '1': 'training'}
 
# Create DataFrame
df = pd.DataFrame(data)    

In [17]:
data = {'0': sub_ids_val,
        '1': 'validation'}
 
# Create DataFrame
df_val = pd.DataFrame(data) 

In [22]:

data = {'0': all_sub_ids,
        '1': 'test'}
 
# Create DataFrame
df_test = pd.DataFrame(data) 

In [23]:
result_df = pd.concat([df,df_val,df_test])

In [28]:
result_df.to_csv('/home/polina/VS_Seg/params/split_schw.csv', header=False, index=False)

In [26]:
check=pd.read_csv('/home/polina/VS_Seg/params/split_schw.csv')

In [27]:
check

,0,VS-SEG-001,training
0,1,VS-SEG-002,training
1,2,VS-SEG-003,training
2,3,VS-SEG-004,training
3,4,VS-SEG-005,training
4,5,VS-SEG-006,training
...,...,...,...
478,237,VS-SEG-246,test
479,238,VS-SEG-247,test
480,239,VS-SEG-248,test
481,240,VS-SEG-249,test


In [20]:
sub_ids = [each.split('/')[-1][:-6] for each in train_imgs]
sub_ids_val = [each.split('/')[-1][:-6] for each in val_imgs_4]
all_sub_ids = [each.split('/')[-1][:-6] for each in imgs]

In [68]:
val_m = metric[metric.Ids.isin(sub_ids)]

In [69]:
val_m.mean()

Unnamed: 0          0.000000
Dice_all                 NaN
Dice_0                   NaN
Dice_1              0.944837
Dice_2              0.941531
Dice_3                   NaN
Hausdorff95_all          NaN
Hausdorff95_0            NaN
Hausdorff95_1       3.465478
Hausdorff95_2            inf
Hausdorff95_4            inf
Sensitivity_all          NaN
Sensitivity_0            NaN
Sensitivity_1       0.936853
Sensitivity_2       0.934720
Sensitivity_4       0.900749
Specificity_all          NaN
Specificity_0            NaN
Specificity_1       0.999607
Specificity_2       0.999863
Specificity_4       0.999853
Surface_dice_all         NaN
Surface_dice_0           NaN
Surface_dice_1      0.912928
Surface_dice_2      0.926532
Surface_dice_4      0.934881
Precision_all            NaN
Precision_0              NaN
Precision_1         0.956309
Precision_2         0.955108
Precision_4         0.910567
Dice_4              0.894122
dtype: float64

In [30]:
metric[metric.Ids==sub_id]

,Unnamed: 0,Ids,Dice_all,Dice_0,Dice_1,Dice_2,Dice_3,Hausdorff95_all,Hausdorff95_0,Hausdorff95_1,...,Surface_dice_0,Surface_dice_1,Surface_dice_2,Surface_dice_4,Precision_all,Precision_0,Precision_1,Precision_2,Precision_4,Dice_4
896,0,BraTS2021_00000,NaN,NaN,0.957302,0.978947,NaN,NaN,NaN,2.236068,...,NaN,0.922439,0.996712,0.995241,NaN,NaN,0.952007,0.96492,0.929315,0.957254


In [25]:
import pandas as pd
metric = pd.read_csv('/home/polina/DeepLearningExamples/PyTorch/Segmentation/nnUNet/notebooks/all_metrics_brats3fold_0.csv')

In [16]:
val_lbls_0 = [each.split('/')[-1] for each in val_lbls_0]

In [18]:
import json
import mpu.io

data = {'fold_0': val_lbls_0,
#         'fold_1': val_lbls_1,
#         'fold_2': val_lbls_2,
#         'fold_3': val_lbls_3,
#         'fold_4': val_lbls_4,
       }
print(json.dumps(data, indent=4))


mpu.io.write('brats_seed.json', data)

{
    "fold_0": [
        "BraTS2021_00000_y.npy",
        "BraTS2021_00009_y.npy",
        "BraTS2021_00016_y.npy",
        "BraTS2021_00024_y.npy",
        "BraTS2021_00028_y.npy",
        "BraTS2021_00031_y.npy",
        "BraTS2021_00035_y.npy",
        "BraTS2021_00045_y.npy",
        "BraTS2021_00046_y.npy",
        "BraTS2021_00051_y.npy",
        "BraTS2021_00070_y.npy",
        "BraTS2021_00078_y.npy",
        "BraTS2021_00085_y.npy",
        "BraTS2021_00087_y.npy",
        "BraTS2021_00088_y.npy",
        "BraTS2021_00089_y.npy",
        "BraTS2021_00099_y.npy",
        "BraTS2021_00102_y.npy",
        "BraTS2021_00104_y.npy",
        "BraTS2021_00106_y.npy",
        "BraTS2021_00107_y.npy",
        "BraTS2021_00110_y.npy",
        "BraTS2021_00117_y.npy",
        "BraTS2021_00123_y.npy",
        "BraTS2021_00126_y.npy",
        "BraTS2021_00127_y.npy",
        "BraTS2021_00130_y.npy",
        "BraTS2021_00136_y.npy",
        "BraTS2021_00137_y.npy",
        "BraTS2021_00140_

{'fold_0': ['BraTS2021_00000_y.npy',
  'BraTS2021_00009_y.npy',
  'BraTS2021_00016_y.npy',
  'BraTS2021_00024_y.npy',
  'BraTS2021_00028_y.npy',
  'BraTS2021_00031_y.npy',
  'BraTS2021_00035_y.npy',
  'BraTS2021_00045_y.npy',
  'BraTS2021_00046_y.npy',
  'BraTS2021_00051_y.npy',
  'BraTS2021_00070_y.npy',
  'BraTS2021_00078_y.npy',
  'BraTS2021_00085_y.npy',
  'BraTS2021_00087_y.npy',
  'BraTS2021_00088_y.npy',
  'BraTS2021_00089_y.npy',
  'BraTS2021_00099_y.npy',
  'BraTS2021_00102_y.npy',
  'BraTS2021_00104_y.npy',
  'BraTS2021_00106_y.npy',
  'BraTS2021_00107_y.npy',
  'BraTS2021_00110_y.npy',
  'BraTS2021_00117_y.npy',
  'BraTS2021_00123_y.npy',
  'BraTS2021_00126_y.npy',
  'BraTS2021_00127_y.npy',
  'BraTS2021_00130_y.npy',
  'BraTS2021_00136_y.npy',
  'BraTS2021_00137_y.npy',
  'BraTS2021_00140_y.npy',
  'BraTS2021_00144_y.npy',
  'BraTS2021_00152_y.npy',
  'BraTS2021_00160_y.npy',
  'BraTS2021_00185_y.npy',
  'BraTS2021_00192_y.npy',
  'BraTS2021_00194_y.npy',
  'BraTS2021_00211

In [ ]:
train_lbls[:5], val_lbls_2[:5], len(train_lbls), len(val_lbls_2)

In [110]:
nfolds=3
data_path = '/data/private_data/schw/33_3d'
kfold = KFold(n_splits=nfolds, shuffle=True, random_state=12345)
fold=2


In [114]:
meta = load_data(data_path, "*_meta.npy")
orig_lbl = load_data(data_path, "*_orig_lbl.npy")
        
imgs, lbls = load_data(data_path, "*_x.npy"), load_data(data_path, "*_y.npy")
train_idx, val_idx = list(kfold.split(imgs))[fold]
orig_lbl, meta = get_split(orig_lbl, val_idx), get_split(meta, val_idx)
train_imgs, train_lbls = get_split(imgs, train_idx), get_split(lbls, train_idx)
val_imgs_2, val_lbls_2 = get_split(imgs, val_idx), get_split(lbls, val_idx)

In [115]:
val_lbls_2 = [each.split('/')[-1].split('.')[0] for each in val_lbls]

In [40]:
# import json
# import mpu.io

# data = {'fold_0': val_lbls_0,
#         'fold_1': val_lbls_1,
#         'fold_2': val_lbls_2,
#        }
# print(json.dumps(data, indent=4))


# mpu.io.write('schw_seed.json', data)

In [144]:
data_path = '/data_anvar/public_datasets/preproc_study/gbm/3a_atlas'
t1_paths = []
t2_paths = []
flair_paths = []
t1c_paths = []
for sub in train_imgs:
    sub = sub.split('/')[-1][:-6]
#     print(sub)
    t1_paths.append(os.path.join(data_path, sub, 'T1.nii.gz'))
    t2_paths.append(os.path.join(data_path, sub, 'T2.nii.gz'))
    t1c_paths.append(os.path.join(data_path, sub, 'CT1.nii.gz'))
    flair_paths.append(os.path.join(data_path, sub, 'FLAIR.nii.gz'))
landmarks_t1_fold2 = tio.HistogramStandardization.train(
    t1_paths,
    output_path='/home/polina/DeepLearningExamples/PyTorch/Segmentation/nnUNet/notebooks/t1_landmarks_fold2.npy',
)
landmarks_t2_fold2 = tio.HistogramStandardization.train(
    t2_paths,
    output_path='/home/polina/DeepLearningExamples/PyTorch/Segmentation/nnUNet/notebooks/t2_landmarks_fold2.npy',
)
landmarks_t1c_fold2 = tio.HistogramStandardization.train(
    t1c_paths,
    output_path='/home/polina/DeepLearningExamples/PyTorch/Segmentation/nnUNet/notebooks/t1c_landmarks_fold2.npy',
)
landmarks_flair_fold2 = tio.HistogramStandardization.train(
    flair_paths,
    output_path='/home/polina/DeepLearningExamples/PyTorch/Segmentation/nnUNet/notebooks/flair_landmarks_fold2.npy',
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:38<00:00,  1.78it/s]


In [145]:
landmarks_dict = {'t1': landmarks_t1_fold2,
                  't2': landmarks_t2_fold2,
                  't1c': landmarks_t1c_fold2,
                  'flair': landmarks_flair_fold2}
histogram_transform = tio.HistogramStandardization(landmarks_dict)

In [146]:
subjects = []
for sub in train_imgs:
    sub = sub.split('/')[-1][:-6]
    subject = tio.Subject(
        t1=tio.ScalarImage(os.path.join(data_path, sub, 'T1.nii.gz')),
        t2=tio.ScalarImage(os.path.join(data_path, sub, 'T2.nii.gz')),
        t1c=tio.ScalarImage(os.path.join(data_path, sub, 'CT1.nii.gz')),
        flair=tio.ScalarImage(os.path.join(data_path, sub, 'FLAIR.nii.gz')),
        label=tio.LabelMap(os.path.join(data_path, sub, 'CT1_SEG.nii.gz')),
    )
    subjects.append(subject)
dataset = tio.SubjectsDataset(subjects)

In [149]:

out_path = '/data/private_data/6_histogram_fold2'
for i ,sample in enumerate(tqdm(dataset)):
        transformed = histogram_transform(sample)
        if not os.path.exists(os.path.join(out_path,str(transformed.t1.path).split('/')[-2])):
            os.mkdir(os.path.join(out_path,str(transformed.t1.path).split('/')[-2]))
        transformed.t1.save(os.path.join(out_path,str(transformed.t1.path).split('/')[-2], 'T1.nii.gz'))
        transformed.t2.save(os.path.join(out_path,str(transformed.t2.path).split('/')[-2], 'T2.nii.gz'))
        transformed.t1c.save(os.path.join(out_path,str(transformed.t1c.path).split('/')[-2], 'CT1.nii.gz'))
        transformed.flair.save(os.path.join(out_path,str(transformed.flair.path).split('/')[-2], 'FLAIR.nii.gz'))
        transformed.label.save(os.path.join(out_path,str(transformed.label.path).split('/')[-2], 'CT1_SEG.nii.gz'))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [12:26<00:00, 10.97s/it]


In [41]:
out_path = '/data/private_data/6_histogram_fold2'
data_path = '/data_anvar/public_datasets/preproc_study/gbm/3a_atlas'
for each in val_imgs:
    sub = each.split('/')[-1][:-6]
    print(sub)
    sub_in = os.path.join(data_path,sub)
    sub_out = os.path.join(out_path,sub)
    if not os.path.exists(sub_out):
            os.system(f'cp -r {sub_in} {sub_out}')  


In [48]:
import json
import os
from glob import glob
from glob2 import glob
from subprocess import call
import time

import nibabel
import numpy as np
from joblib import Parallel, delayed


def load_nifty(directory, example_id, suffix):
    return nibabel.load(f'{directory}/{suffix}.nii.gz')


def load_channels(d, example_id):
    return [load_nifty(d, example_id, suffix) for suffix in ["FLAIR", "T1", "CT1", "T2"]]


def get_data(nifty, dtype="int16"):
    if dtype == "int16":
        data = np.abs(nifty.get_fdata().astype(np.int16))
        data[data == -32768] = 0
        return data
    return nifty.get_fdata().astype(np.uint8)


def prepare_nifty(d, d_out):
    sub = d.split("/")[-1]
    flair, t1, t1ce, t2 = load_channels(d, sub)
    affine, header = flair.affine, flair.header
    vol = np.stack([get_data(flair), get_data(t1), get_data(t1ce), get_data(t2)], axis=-1)
    vol = nibabel.nifti1.Nifti1Image(vol, affine, header=header)
#     print('l')
#     print( os.path.join(d_out, sub + ".nii.gz"))
    nibabel.save(vol, os.path.join(d_out, sub + ".nii.gz"))
#     print(glob(f'{d}/{sub}/**/MASK.nii.gz')[0])
    if os.path.exists(f'{d}/CT1_SEG.nii.gz'):
        seg = load_nifty(d, sub, "CT1_SEG")
        affine, header = seg.affine, seg.header
        vol = get_data(seg, "unit8")
#         vol[vol == 4] = 3
#         mask = np.zeros(vol.shape, dtype=np.uint8)
#         mask[vol == 2] = 1
#         mask[vol == 3] = 2
#         mask[vol == 1] = 3
        seg = nibabel.nifti1.Nifti1Image(vol, affine, header=header)
#         print(os.path.join(d_out, sub + "_seg.nii.gz"))
        nibabel.save(seg, os.path.join(d_out, sub + "_seg.nii.gz"))


def prepare_dirs(d_out, train):
    img_path, lbl_path = os.path.join(d_out, "images"), os.path.join(d_out, "labels")
    call(f"mkdir {img_path}", shell=True)
    if train:
        call(f"mkdir {lbl_path}", shell=True)
    dirs = glob(os.path.join(d_out, "*"))
    for d in dirs:
        if '.nii.gz' in d:
                if "FLAIR" in d or "T1" in d or "CT1" in d or "T2" in d:
                    continue
                if "_seg" in d:
                    call(f"mv {d} {lbl_path}", shell=True)
                else:
                    call(f"mv {d} {img_path}", shell=True)
                
#         call(f"rm -rf {d}", shell=True)
         

def prepare_dataset_json(d_out, train):
    images, labels = glob(os.path.join(d_out, "images", "*")), glob(os.path.join(d_out, "labels", "*"))
    images = sorted([img.replace(d_out + "/", "") for img in images])
    labels = sorted([lbl.replace(d_out + "/", "") for lbl in labels])
    
    modality = {"0": "FLAIR", "1": "T1", "2": "T1CE", "3": "T2"}
    labels_dict = {"0": "background", "1": "edema", "2": "tumor core", "3": "enhancing tumour"}
    if train:
        key = "training"
        data_pairs = [{"image": img, "label": lbl} for (img, lbl) in zip(images, labels)]
    else:
        key = "test"
        data_pairs = [{"image": img} for img in images]

    dataset = {
        "labels": labels_dict,
        "modality": modality,
        key: data_pairs,
    }

    with open(os.path.join(d_out, "dataset.json"), "w") as outfile:
        json.dump(dataset, outfile)


def run_parallel(func, args):
    return Parallel(n_jobs=os.cpu_count())(delayed(func)(arg) for arg in args)


def prepare_dataset(data, train, out):
    print(f"Preparing GBM dataset from: {data}")
    d_out = out
    if not os.path.exists(d_out):
        call(f"mkdir {d_out}", shell=True)
    start = time.time()
#     run_parallel(prepare_nifty, sorted(glob(os.path.join(data, "*"))))
    for each in sorted(glob(os.path.join(data, "*"))):
        if os.path.exists(os.path.join(each, "CT1_SEG.nii.gz")):
            prepare_nifty(each, d_out)
    prepare_dirs(d_out, train)
    prepare_dataset_json(d_out, train)
    end = time.time()
    print(f"Preparing time: {(end - start):.2f}")


In [49]:
prepare_dataset('/data_anvar/public_datasets/preproc_study/gbm/6_hist/6_hist_fold_1',True, '/data/private_data/6_histogram_fold_1' )
# prepare_dataset('/data_anvar/public_datasets/preproc_study/gbm/7a_resamp',True, '/data/private_data/7a_resample')

Preparing GBM dataset from: /data_anvar/public_datasets/preproc_study/gbm/6_hist/6_hist_fold_1
Preparing time: 365.88


In [47]:
!cd /data_anvar/public_datasets/preproc_study/gbm/6_hist/6_hist_fold_1/TCGA-02-0006 && ls

CT1.nii.gz  CT1_SEG.nii.gz  FLAIR.nii.gz  T1.nii.gz  T2.nii.gz


In [50]:
!python3 ../preprocess.py --data /data/private_data --task 31.1 --ohe --exec_mode training --results /data/private_data/

Preprocessing /data/private_data/6_histogram_fold_1
Pre-processing time: 72.37


In [34]:
len(os.listdir('/data_anvar/public_datasets/preproc_study/gbm/6_hist/6_hist_fold_1'))

102

In [36]:
len(os.listdir('/data/private_data/31.0_3d'))

409

In [37]:
len(os.listdir('/data/private_data/31.1_3d'))

409

In [38]:
# !cd /data/private_data/31.2_3d/ && ls

In [70]:
!cd /results/gbm_results/6_histogram_fold_1/fold-1/checkpoints && ls -l

total 5363204
-rw-r--r-- 1 root root 610204723 Mar  1 12:12 'best_epoch=0-dice_mean=6.54.ckpt'
-rw-r--r-- 1 root root 610206063 Mar  1 12:54 'best_epoch=27-dice_mean=57.69.ckpt'
-rw-r--r-- 1 root root 610206063 Mar  1 12:37 'epoch=14-dice_mean=53.15.ckpt'
-rw-r--r-- 1 root root 610206063 Mar  1 12:44 'epoch=19-dice_mean=54.09.ckpt'
-rw-r--r-- 1 root root 610206063 Mar  1 12:51 'epoch=24-dice_mean=55.33.ckpt'
-rw-r--r-- 1 root root 610206063 Mar  1 12:57 'epoch=29-dice_mean=57.60.ckpt'
-rw-r--r-- 1 root root 610206063 Mar  1 12:20 'epoch=4-dice_mean=34.75.ckpt'
-rw-r--r-- 1 root root 610206063 Mar  1 12:30 'epoch=9-dice_mean=44.89.ckpt'
-rw-r--r-- 1 root root 610206063 Mar  1 13:01  last.ckpt


In [38]:
len(os.listdir('/data_anvar/public_datasets/preproc_study/gbm/6_hist/6_histogram/6_hist_fold_0'))

102

In [75]:
import torch 
ch = torch.load('/results/gbm_results/6_histogram_fold_1/fold-1/checkpoints/last.ckpt')

In [52]:
for i in ch:
    print(i)

epoch
global_step
pytorch-lightning_version
state_dict
callbacks
optimizer_states
lr_schedulers
native_amp_scaling_state
hparams_name
hyper_parameters


In [76]:
ch['epoch']

92

In [53]:
ch['hyper_parameters']

{'args': Namespace(affinity='socket_unique_contiguous', amp=True, batch_size=2, benchmark=False, blend='gaussian', brats=True, ckpt_path=None, data='/data/private_data/31.1_3d', data2d_dim=3, deep_supervision=True, deep_supr_num=2, depth=6, dim=3, epochs=100, exec_mode='train', filters=[64, 96, 128, 192, 256, 384, 512], focal=False, fold=1, gpus=1, gradient_clip_val=0, invert_resampled_y=False, learning_rate=0.0003, logname=None, min_fmap=2, momentum=0.99, more_chn=False, negative_slope=0.01, nfolds=3, no_back_in_output=False, norm='instance', num_workers=8, nvol=4, optimizer='adam', overlap=0.5, oversampling=0.4, patience=100, profile=False, res_block=False, results='/results/gbm_results/6_histogram_fold_1/fold-1', resume_training=False, save_ckpt=True, save_preds=False, scheduler=True, seed=1, skip_first_n_eval=0, sync_batchnorm=False, task='31.1', test_batches=0, train_batches=0, tta=False, val_batch_size=4, warmup=5, weight_decay=0.0001),
 'triton': False,
 'data_dir': None}